#### Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
basic_url = "https://datasets.imdbws.com/title.basics.tsv.gz"
akas_url = "https://datasets.imdbws.com/title.akas.tsv.gz"
ratings_url = "https://datasets.imdbws.com/title.ratings.tsv.gz"

#### Ratings file

In [3]:
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1910
1,tt0000002,5.8,256
2,tt0000003,6.5,1715
3,tt0000004,5.6,169
4,tt0000005,6.2,2529


#### Akas File

In [4]:
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)

In [5]:
akas.replace({'\\N':np.nan}, inplace=True)
akas

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,NaN,imdbDisplay,NaN,0
1,tt0000001,2,Carmencita,DE,NaN,NaN,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,NaN,imdbDisplay,NaN,0
3,tt0000001,4,Καρμενσίτα,GR,NaN,imdbDisplay,NaN,0
4,tt0000001,5,Карменсита,RU,NaN,imdbDisplay,NaN,0
...,...,...,...,...,...,...,...,...
33243175,tt9916852,5,Episódio #3.20,PT,pt,NaN,NaN,0
33243176,tt9916852,6,Episodio #3.20,IT,it,NaN,NaN,0
33243177,tt9916852,7,एपिसोड #3.20,IN,hi,NaN,NaN,0
33243178,tt9916856,1,The Wind,DE,NaN,imdbDisplay,NaN,0


In [6]:
#Filter for us only
akas = akas[akas['region'] == 'US']
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


#### Basics File

In [7]:
basics_main = pd.read_csv(basic_url, sep='\t', low_memory=False)

In [8]:
#copy for faster loading
basics = basics_main.copy()
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [9]:
basics.replace({'\\N':np.nan}, inplace=True)

In [10]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9237277 entries, 0 to 9237276
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 634.3+ MB


In [11]:
#drop missing start year
#drop runtimeMinsutes null
#drop missing genre
basics.dropna(subset=['runtimeMinutes', 'genres', 'startYear'], inplace=True)

In [12]:
basics['startYear'] = basics['startYear'].astype(int)
basics = basics[basics['startYear'] >=2000]
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
13079,tt0013274,movie,Istoriya grazhdanskoy voyny,Istoriya grazhdanskoy voyny,0,2021,NaN,133,Documentary
33792,tt0034413,short,Youth Gets a Break,Youth Gets a Break,0,2001,NaN,20,Short
34792,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
39534,tt0040241,short,Color Rhapsodie,Color Rhapsodie,0,2021,NaN,6,Short
43538,tt0044326,short,Abstronic,Abstronic,0,2021,NaN,6,Short


In [13]:
#keep only movie type
basics = basics[basics['titleType'] == 'movie']

In [14]:
basics.isna().sum()

tconst                 0
titleType              0
primaryTitle           0
originalTitle          0
isAdult                0
startYear              0
endYear           216533
runtimeMinutes         0
genres                 0
dtype: int64

In [15]:
#drop documentary genres
is_doc = basics['genres'].str.contains('documentary', case=False)
basics = basics[~is_doc]
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34792,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61093,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
67639,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
77933,tt0079644,movie,November 1828,November 1828,0,2001,NaN,140,"Drama,War"
86770,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"


In [16]:
basics.shape

(143474, 9)

In [17]:
#filter df based on another df to keep only us movies
keepers = basics['tconst'].isin(akas['titleId'])
basics = basics[keepers]
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34792,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61093,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
67639,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
86770,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
93906,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama


In [18]:
basics.shape

(82820, 9)

#### Saving dataframes

In [19]:
#makig new folder with os
import os
os.makedirs('Data/', exist_ok=True)
#confirm folder is created
os.listdir('Data/')

['.DS_Store', 'ratings.csv.gz', 'title_basics.csv.gz', 'title_akas.csv.gz']

In [20]:
#save current dataframe to file
basics.to_csv('Data/title_basics.csv.gz', compression='gzip', index=False)
akas.to_csv('Data/title_akas.csv.gz', compression='gzip', index=False)
ratings.to_csv('Data/ratings.csv.gz', compression='gzip', index=False)

In [21]:
#open saved file and preview again
basics = pd.read_csv('Data/title_basics.csv.gz', low_memory=False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama


#### DF .info()

In [22]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82820 entries, 0 to 82819
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          82820 non-null  object 
 1   titleType       82820 non-null  object 
 2   primaryTitle    82820 non-null  object 
 3   originalTitle   82820 non-null  object 
 4   isAdult         82820 non-null  int64  
 5   startYear       82820 non-null  int64  
 6   endYear         0 non-null      float64
 7   runtimeMinutes  82820 non-null  int64  
 8   genres          82820 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 5.7+ MB


In [23]:
akas = pd.read_csv('Data/title_akas.csv.gz', low_memory=False)
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1351235 entries, 0 to 1351234
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   titleId          1351235 non-null  object 
 1   ordering         1351235 non-null  int64  
 2   title            1351235 non-null  object 
 3   region           1351235 non-null  object 
 4   language         3704 non-null     object 
 5   types            964195 non-null   object 
 6   attributes       45010 non-null    object 
 7   isOriginalTitle  1349860 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 82.5+ MB


In [24]:
ratings = pd.read_csv('Data/ratings.csv.gz', low_memory=False)
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1261423 entries, 0 to 1261422
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1261423 non-null  object 
 1   averageRating  1261423 non-null  float64
 2   numVotes       1261423 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 28.9+ MB
